In [ ]:
%matplotlib inline

import subprocess
import os

import json
import pandas as pd

import numpy as np 
import matplotlib.pyplot as plt

from scipy.io import loadmat

from owi.visualization import matplotlib_helpers
from owi.data_scans import utils_data

from owi.data_scans import CONFIG_FILENAME

# dir_path = os.path.dirname(os.path.realpath(__file__))
# CONFIG_FILENAME = os.path.join(dir_path, 'config_ipynb')

# Load Scan data

- Mat file:
    - processedValueMap3D is the signal from the ROI
    - referenceVoltageMap3D is the voltage form the photodiode reading from the reference beam (after AOM's)
    - objectVoltageMap3D is the photodiode voltage read from the laser beam.

In [ ]:
with open(CONFIG_FILENAME,'r') as f:
    f_folder = f.read()
    
print('Data folder: %s'%f_folder)

In [ ]:
f_dict = utils_data.get_data_files(f_folder)

f_mat = f_dict['f_mat']
f_json = f_dict['f_meta']
f_csv = f_dict['f_csv']

f_dict

In [ ]:
f_data = loadmat(f_mat)
f_data.keys()

In [ ]:
inten_meas = f_data['processedValueMap3D']
inten_laser = f_data['objectVoltageMap3D']
inten_reference = f_data['referenceVoltageMap3D']

In [ ]:
inten_meas.shape

In [ ]:
with open(f_json) as f:
    d_json = json.load(f)

In [ ]:
d_json

In [ ]:
plt.figure()
plt.plot(inten_meas.ravel())
plt.title('Raw ROI processed value from 3D array')

# plt.figure()
# plt.plot(inten_laser.ravel())

# Plot in 3D

In [ ]:
from mayavi import mlab 
from owi.visualization import mayavi_utils
# mlab.init_notebook()

## Generate 3D corrdinates

From JSON file

In [ ]:
# xyz_3d = np.mgrid[0:d_json['XMax (mm)']+d_json['XStepSize (mm)']:d_json['XStepSize (mm)'], 
#          0:d_json['YMax (mm)']:d_json['YStepSize (mm)'],
#          0:d_json['ZMax (mm)']+d_json['ZStepSize (mm)']:d_json['ZStepSize (mm)']]

shape = inten_meas.shape
if 'XMax (mm)' in d_json:
    xyz_max = [d_json['XMax (mm)'], d_json['YMax (mm)'], d_json['ZMax (mm)'], ]
elif 'scanParameters' in d_json:
    if 'xMax_mm' in d_json['scanParameters']:
        xyz_max = [d_json['scanParameters']['xMax_mm'], 
                   d_json['scanParameters']['yMax_mm'], 
                   d_json['scanParameters']['zMax_mm'], ]
    else:
        xyz_max = [d_json['scanParameters']['xLength_mm'], 
                   d_json['scanParameters']['yLength_mm'], 
                   d_json['scanParameters']['zLength_mm'], ]

xyz_3d = np.mgrid[0:xyz_max[0]:shape[0]*1j, 
             0:xyz_max[1]:shape[1]*1j,
             0:xyz_max[2]:shape[2]*1j]

In [ ]:
xyz_3d.shape, inten_meas.shape

In [ ]:
mlab.figure(size=(800,700))

In [ ]:
mlab.clf()
mlab.contour3d(xyz_3d[0], xyz_3d[1], xyz_3d[2], inten_meas, contours=10, opacity=.5)
mlab.colorbar()
mlab.axes()

mayavi_utils.inline(stop=False)

eng = mlab.get_engine()
scene = eng.scenes[0]
scene.scene.parallel_projection = True

scene.scene.x_minus_view()
mayavi_utils.inline(stop=False)

scene.scene.y_minus_view()
mayavi_utils.inline(stop=False)

scene.scene.z_minus_view()
mayavi_utils.inline(stop=False)

In [ ]:
mlab.clf()
mlab.contour3d(xyz_3d[0], xyz_3d[1], xyz_3d[2], -inten_meas, contours=10, opacity=.5)
mlab.colorbar()
mlab.axes()

mayavi_utils.inline(stop=False)

eng = mlab.get_engine()
scene = eng.scenes[0]
scene.scene.parallel_projection = True

scene.scene.x_minus_view()
mayavi_utils.inline(stop=False)

scene.scene.y_minus_view()
mayavi_utils.inline(stop=False)

scene.scene.z_minus_view()
mayavi_utils.inline(stop=False)

In [ ]:
fig, axs = matplotlib_helpers.plot_projections(xyz_3d, inten_meas, proj_fn='min', unit_str=' mm',)
fig.set_size_inches(7,12)

In [ ]:
fig, axs = matplotlib_helpers.plot_projections(xyz_3d, inten_meas, proj_fn='max', unit_str=' mm',)
fig.set_size_inches(7,12)

# Load list mode data

This data contains a list of all measurement points and positions of the ultrasound. 

### Plot raw values

In [ ]:
if f_csv is not None:
    d_listmode = pd.read_csv(f_csv)
    d_listmode.plot(subplots=True, figsize=(10,10));
    d_listmode[:10]
else:
    print("WARNING: no CSV file")

# Plot all layers
This will show all the different z-planes for this scan

In [ ]:
folder_render = 'generated'
try:
    os.mkdir(folder_render)
except FileExistsError:
    pass

In [ ]:
n_layer = inten_meas.shape[2]

for i in range(n_layer):
    plt.figure()
    plt.pcolormesh(xyz_3d[0,:,:,0], xyz_3d[1,:,:,0], inten_meas[:,:,i], 
                   vmax=inten_meas.max(), vmin=inten_meas.min(), 
                  cmap='bone')
    plt.colorbar()
    plt.title('z ind = %d'%(i))
    plt.xlabel('x mm')
    plt.ylabel('y mm')
    plt.savefig(folder_render + '/testFig%d.png'%i)
    plt.savefig(folder_render + '/testFig%d.png'%(2*n_layer-i-1))

In [ ]:
# generate animations
call_str = 'ffmpeg -y -r 8 -i {folder}/testFig%d.png {folder}/anim_{}.gif -b:v 10M'.format(f_folder.split('/')[-2], folder=folder_render)
out = subprocess.call(call_str.split())

call_str = 'ffmpeg -y -r 8 -i {folder}/testFig%d.png -vcodec libx264 -crf 25 -pix_fmt yuv420p {folder}/anim_{}.mp4'.format(f_folder.split('/')[-2], folder=folder_render)
out = subprocess.call(call_str.split())

